## 导入数据包

In [1]:
import pandas as pd
import numpy as np
import os

## 修改主目录

In [6]:
os.chdir(r'C:/Users/yves1/Desktop/')

## 基础函数方法

In [7]:
 # 读取SKU配置文件信息
def read_sku():
    return pd.read_json('../DataSet/sku_data.json')

# 获取文件名称
def find_file_name(name_start):
    for item in os.listdir():
        if item.startswith(name_start):
            return item

# 通过ASIN信息查询SKU信息
def parse_sku(ASIN):
    sku_data = read_sku()
    return pd.Series([sku_data.loc[ASIN.strip(), 'SKU']])

## 读取Business Report数据

In [4]:
def read_business_report(date_value):
    business_data = pd.read_csv(find_file_name('BusinessReport'))
    business_data.drop(['Title', 'Session Percentage', 'Page Views Percentage', 'Units Ordered - B2B', 'Unit Session Percentage - B2B', 'Ordered Product Sales - B2B', 'Total Order Items - B2B'], axis=1, inplace=True)
    business_data['Date'] = date_value
    business_data = business_data.combine_first(business_data['(Child) ASIN'].apply(parse_sku))
    business_data.rename(columns={0: 'SKU', '(Parent) ASIN': 'Parent', '(Child) ASIN': 'Child', 'Buy Box Percentage': 'Buy Box', 'Unit Session Percentage': 'CR', 'Ordered Product Sales': 'Sales', 'Total Order Items': 'Order Items'}, inplace=True)
    business_data.sort_values(by='SKU', ascending=True, inplace=True)
    business_data = business_data[['Date', 'Parent', 'Child', 'SKU', 'Sessions', 'Page Views', 'Order Items', 'Units Ordered', 'Sales', 'CR', 'Buy Box']]
    return business_data

## 读取Advertisement Report数据

In [5]:
def read_sponsored_report(date_value):
    sponsored_data = pd.read_excel(find_file_name('Sponsored Products'))
    sponsored_data = sponsored_data.query('Date==["{}"]'.format(date_value))
    sponsored_data = sponsored_data[['Date', 'Campaign Name', 'Targeting', 'Match Type', 'Customer Search Term', 'Impressions', 'Clicks', 'Cost Per Click (CPC)', 'Spend', '7 Day Total Sales ', 'Click-Thru Rate (CTR)', '7 Day Conversion Rate', 'Total Advertising Cost of Sales (ACoS) ']].sort_values(by=['Date', 'Campaign Name'], ascending=False)
    sponsored_data.rename(columns={'Customer Search Term': 'Search Term', 'Cost Per Click (CPC)': 'CPC', '7 Day Total Sales ': 'Sales', 'Click-Thru Rate (CTR)': 'CTR', '7 Day Conversion Rate': 'CR', 'Total Advertising Cost of Sales (ACoS) ': 'ACOS'}, inplace=True)
    sponsored_data['ACOS'].fillna(value=0,inplace=True)
    return sponsored_data

## 数据导出

In [6]:
def export_data(date_value):
    file_name = 'data_{}.xlsx'.format(date_value.replace('/', ''))
    writer = pd.ExcelWriter(file_name)
    read_business_report(date_value).to_excel(writer, sheet_name='Business Report', index=False)
    read_sponsored_report(date_value).to_excel(writer, sheet_name='Search Term Report', index=False)
    writer.save()

## 执行操作

In [10]:
export_data('2020/12/18')